In [ ]:
from k_means_constrained import KMeansConstrained
import pandas as pd
import geopandas
import numpy as np
import os
import openrouteservice as ors

In [ ]:
# Get your ORS 
passcode = ' '

In [ ]:
folder = 'demo'
file = 'plan.xlsx'
path = os.path.join(folder, file)
df = pd.read_excel('plan.xlsx')

In [ ]:
# Setting the cluster minimum and maximum size
min_size = 15
max_size = 18

# Finding the total delivery points, clustering algorithm accept only X and Y value in numpy format
# Hence extracting the latitude and longitude information and converting the DataFrame as numpy.  
latlon = df[['Longitude', 'Latitude']].to_numpy()
total = latlon.shape[0]

# Finding the maximum total cluster. 
total_clusters = int(total/max_size) + 1

clusters = KMeansConstrained(n_clusters=total_clusters,
                               size_min=min_size, 
                               size_max=max_size,
                              random_state=0)
clusters_values  = clusters.fit_predict(latlon)


In [ ]:
# write cluster numbers to GeoDataFrame

df['cluster'] = clusters_values

# Create a new column return and set it to true 1. 
# Dispatcher can modify this based on his needs. 

df['toWH'] = 1

# Create GeoDataFrame
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))


# Write the GeoDataFrame as GeoPackage. 
folder = 'demo'
file = 'clusters.gpkg'
path = os.path.join(date, folder)
gdf.to_file('clusters.gpkg', layer='locations', driver="GPKG")

In [ ]:
#warehouse

wh = [lon,lat]

def route_optimize(cluster):
    vehicles = []
    deliveries = []
    cluster_df = df[df['cluster']==cluster]

    return_condition = list(cluster_df['toWH'])[0]

    for delivery in cluster_df.itertuples():
            deliveries.append(ors.optimization.Job(id=delivery.Index, location=[delivery.Longitude, delivery.Latitude]))
    
    # Set vechile condition based on the return condition 
    if return_condition:
        vehicles.append(ors.optimization.Vehicle(id=0,start=list(wh), end=list(wh)))
    else:
        vehicles.append(ors.optimization.Vehicle(id=0,start=list(wh)))
    
    ors_client = ors.Client(key=passcode)
    result = ors_client.optimization(jobs=deliveries, vehicles=vehicles, geometry=True)
    
    li = result['routes'][0]['steps']
    li2 = []
    for n, x in enumerate(li):
        lon = x['location'][0]
        lat = x['location'][1]
        li2.append([n, lat, lon])
        
    order_df = pd.DataFrame(li2, columns=['order', 'lon', 'lat'])
    
    file_name = 'route_' + str(int(return_condition)) + '.xlsx'
    path_excel = os.path.join(folder, file_name)
    order_df.to_excel(path_excel)
    return path_excel

In [ ]:
file_name = 'clusters.gpkg'
path_df = os.path.join(folder, file_name)
df = geopandas.read_file(path_df, layer='locations')

#Get total clusters
total_cluster =  list(df['cluster'].unique())

for z in total_cluster:
    print(route_optimize(z), ' file written successfully')